In [1]:
import tkinter as tk
from PIL import ImageTk, Image
import numpy as np
import colour as color

In [2]:
# Represents a 3D Vector in space with a color
class Vector:
    def __init__(self, x, y, z, color = "red"):
        self.x = x
        self.y = y
        self.z = z
        self.color = color

        # Matrix representation of the Vector
        self.matrix = np.array([x, y, z])

    # Adds two vectors
    def __add__(self, vector):
        return Vector(self.x + vector.x, self.y + vector.y, self.z + vector.z, vector.color)

    # Substracts two vectors
    def __sub__(self, vector):
        return Vector(self.x - vector.x, self.y - vector.y, self.z - vector.z, vector.color)

    # Displays the Vector information as a string when printed
    def __str__(self):
        return f"Vector({self.x}, {self.y}, {self.z}, {self.color})"

    def magnitude(self):
        return (self.x ** 2 + self.y ** 2 + self.z ** 2) ** 0.5

    # Returns the Vector with the same direction b a magnitude of 1
    def normalized(self):
        return Vector(self.x / self.magnitude(), self.y / self.magnitude(), self.z / self.magnitude(), self.color)

    # Returns the vector perpendicular to this and another vector
    def cross_product(self, vector):
        return Vector(self.y * vector.z - self.z * vector.y,
                    self.z * vector.x - self.x * vector.z,
                    self.x * vector.y - self.y * vector.x, 0)
    
    def dot_product(self, vector):
        return np.dot(self.np_matrix(), vector.np_matrix())
    
    def norm_product(self, vector):
        return np.linalg.norm(self.np_matrix() * np.linalg.norm(vector.np_matrix()))

    def np_matrix(self):
        return np.array([self.x, self.y, self.z])
    
    # Returns the angle between this vector and another
    def angle(self, vector):
        return np.arccos(self.dot_product(vector) / self.norm_product(vector))
    
    # Draws the point on screen
    def draw_vertex(self, screen):
        screen.draw_Vector(self, 5)

In [3]:
a = Vector(1, 0, 0)
b = Vector(0, 1, 0)

In [4]:
# Represents a 3D polygon in space with 3 vertices and a base color
class Polygon:
    def __init__(self, v1, v2, v3, color):
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3
        self.color = color
        self.normal = self.calculate_normal()

    # Displays the Polygon information as a string when printed
    def __str__(self):
        return f"Polygon({self.v1}, {self.v2}, {self.v3}, {self.color})"

    # Returns the normal of the polygon
    def calculate_normal(self):
        return self.v1.cross_product(self.v2).normalized()

In [5]:
# Represents the properties and transformation matrices of a camera
class Camera:
    def __init__(self, u, v, w, origin, height, width, focus, far_plane, close_plane):
        self.u = u
        self.v = v
        self.w = w
        self.origin = origin
        self.height = height
        self.width = width
        self.focus = focus
        self.far_plane = far_plane
        self.close_plane = close_plane

        # Aspect ratio of the camera
        self.aspect_ratio = width / height

        # Matrix representation of the camera
        self.camera_matrix = np.array([
            [self.u.x, self.v.x, self.w.x],
            [self.u.y, self.v.y, self.w.y],
            [self.u.z, self.v.z, self.w.z]
        ])

        # Transposed matrix representation of the camera
        self.transposed_camera_matrix = np.transpose(self.camera_matrix)

        # Matrix representation of the origin
        self.origin_matrix = -np.matmul(self.transposed_camera_matrix, self.origin.matrix)

        # Transformation matrix of the camera
        self.transformation_matrix = np.array([
            [self.u.x, self.u.y, self.u.z, self.origin_matrix[0]],
            [self.v.x, self.v.y, self.v.z, self.origin_matrix[1]],
            [self.w.x, self.w.y, self.w.z, self.origin_matrix[2]],
            [0, 0, 0, 1]
        ])

        # Projection matrix of the camera
        self.projection_matrix = np.array([
            [self.focus / self.width, 0, 0, 0],
            [0, self.focus / self.height, 0, 0],
            [0, 0, -(self.far_plane + self.close_plane) / (self.far_plane - self.close_plane), -2 * self.far_plane * self.close_plane / (self.far_plane - self.close_plane)],
            [0, 0, -1, 0]
        ]) 

In [6]:
# Represents screen space and displays the objects on it
class Screen:
    def __init__(self, width, height, window, camera, light_model):
        self.width = width
        self.height = height
        self.window = window
        self.camera = camera
        self.light_model = light_model

        # Calculates the delta between the center of the screen and the center of the window
        self.x_delta = (width - 1) / (camera.width * 2)
        self.y_delta = (height - 1) / (camera.height * 2)

        # Sets up the window
        self.window.geometry(str(width) + "x" + str(height))
        self.canvas = tk.Canvas(self.window, width = width, height = height)    
        self.canvas.configure(background = "black")
        self.canvas.pack()

        # Displays lights
        for light in self.light_model.point_lights:
            self.draw_Vector(light, 5)


    # PRIVATE VARIABLES
    # Transforms a Vector in world space (x, y, z) to projection space (x, y, z, w)
    def _world_to_projection(self, Vector):
        Vector_4d = np.append(Vector.matrix, 1)
        transformed = np.matmul(self.camera.transformation_matrix, Vector_4d)
        projected = np.matmul(self.camera.projection_matrix, transformed)
        projected_scaled = projected / projected.max()
        return projected_scaled
    
    # Transforms a Vector in world space (x, y, z) to screen coordinates (x, y)
    def _world_to_screen_coordinate(self, Vector):
        Vector_2d = self._world_to_projection(Vector)
        display_x = (self.x_delta * Vector_2d[0]) + (self.x_delta * self.camera.width) + self.camera.width
        display_y = (self.y_delta * Vector_2d[1]) + (self.y_delta * self.camera.height) + self.camera.height
        return [round(display_x), round(display_y)]
    
    # Calculates the steps between two coordinates in screen space (x, y) and the delta in each direction
    def _calculate_pixel_steps(self, a, b):
        # Calculates the difference between the Vectors
        delta_x = a[0] - b[0]
        delta_y = a[1] - b[1]

        # Calculates the number of steps needed to draw the line on the larger axis and scales the delta accordingly
        if abs(delta_x) > abs(delta_y):
            steps = abs(delta_x)
            delta_y = delta_y / steps
            delta_x = delta_x / steps
        else:
            steps = abs(delta_y)
            delta_y = delta_y / steps
            delta_x = delta_x / steps

        return steps, -delta_x, -delta_y
    

    # PUBLIC VARIABLES
    # Draws a circle on the screen with the given radius and color from a Vector in world space (x, y, z)
    def draw_Vector(self, Vector, size = 1):
        screen_coordinate = self._world_to_screen_coordinate(Vector)
        self.canvas.create_oval(screen_coordinate[0] - size, screen_coordinate[1] - size, screen_coordinate[0] + size, screen_coordinate[1] + size, fill = Vector.color, outline = Vector.color)

    # Draws a pixel on a screen coordinate (x, y) with the given color
    def draw_coordinate(self, x, y, color = "white"):
        self.canvas.create_oval(x, y, x, y, fill = color, outline = color)

    # Creates a list of screen coordinates (x, y) between two Vectors in world space (x, y, z)
    def world_line_coordinates(self, a, b):
        # Converts Vectors from world space to screen space
        coordinate_a = self._world_to_screen_coordinate(a)
        coordinate_b = self._world_to_screen_coordinate(b)

        # Calculates the number of steps and delta in each direction
        steps, delta_x, delta_y = self._calculate_pixel_steps(coordinate_a, coordinate_b)

        # Creates the list of coordinates
        line_coordinates = []
        for i in range(steps + 1):
            line_coordinates.append((round(coordinate_a[0] + i * delta_x), round(coordinate_a[1] + i * delta_y)))

        return line_coordinates
    
    # Draws a line in the screen between two Vectors in world space (x, y, z)
    def draw_line_between_Vectors(self, a, b):
        coordinates = self.world_line_coordinates(a, b)
        for i in range(len(coordinates)):
            self.draw_coordinate(coordinates[i][0], coordinates[i][1])

In [7]:
# Represents the illumination setup of the scene
class LightModel:
    def __init__(self, ambient_intensity):
        self.ambient_intensity = ambient_intensity
        self.point_lights = []

    # Adds a point light to the scene as a Vector in world space (x, y, z)
    def add_point_light(self, Vector):
        self.point_lights.append(Vector)

In [8]:
# Represents a 3D object in space
class Cube:
    def __init__(self, size):
        self.size = size
        self.vertices = self._calculate_vertices()

    # Calculates the vertices of the cube
    def _calculate_vertices(self):
        vertices = []
        offset = self.size - self.size/2
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    vertices.append(Vector(x * self.size - offset, y * self.size - offset, z * self.size - offset))
        return vertices
    
    # Draws the vertices of the cube on screen
    def draw_vertices(self, screen):
        for vertex in self.vertices:
            screen.draw_Vector(vertex, 5)
    
    # Draws the wireframe of the cube on screen
    def draw_wireframe(self, screen):
        screen.draw_line_between_Vectors(self.vertices[4], self.vertices[0])
        screen.draw_line_between_Vectors(self.vertices[4], self.vertices[5])
        screen.draw_line_between_Vectors(self.vertices[0], self.vertices[1])
        screen.draw_line_between_Vectors(self.vertices[5], self.vertices[1])

        screen.draw_line_between_Vectors(self.vertices[4], self.vertices[6])
        screen.draw_line_between_Vectors(self.vertices[0], self.vertices[2])
        screen.draw_line_between_Vectors(self.vertices[5], self.vertices[7])
        screen.draw_line_between_Vectors(self.vertices[1], self.vertices[3])

        screen.draw_line_between_Vectors(self.vertices[6], self.vertices[7])
        screen.draw_line_between_Vectors(self.vertices[6], self.vertices[2])
        screen.draw_line_between_Vectors(self.vertices[2], self.vertices[3])
        screen.draw_line_between_Vectors(self.vertices[7], self.vertices[3])

        screen.draw_line_between_Vectors(self.vertices[4], self.vertices[1])
        screen.draw_line_between_Vectors(self.vertices[5], self.vertices[3])
        screen.draw_line_between_Vectors(self.vertices[7], self.vertices[2])
        screen.draw_line_between_Vectors(self.vertices[6], self.vertices[0])
        screen.draw_line_between_Vectors(self.vertices[4], self.vertices[7])
        screen.draw_line_between_Vectors(self.vertices[1], self.vertices[2])

In [9]:
# Create the main window
window = tk.Tk()
window.title("Phong Illumination Model")

''

In [10]:
# Camera declaration
u = Vector(0, 0, 1)
v = Vector(0, -1, 0)
w = Vector(-1, 0, 0)
origin = Vector(-4, 1.5, 1.8)
cam = Camera(u, v, w, origin, 1, 1, 1, 12, 3)

# Light declaration
lights = LightModel(0.5)
lights.add_point_light(Vector(2, 2, 2, color.Color("white")))

# Scren declaration
screen = Screen(500, 500, window, cam, lights)

# Vertex declaration (blue)
a = Vector(2, 0, 0, color.Color("blue"))
a.draw_vertex(screen)

# Normal declaration (yellow)
normal = Vector(1, 0, 0, color.Color("yellow"))
a_normal = a + normal
a_normal.draw_vertex(screen)

# Light vector declaration (green)
light = (lights.point_lights[0] - a).normalized()
light.color = color.Color("green")
print("light:")
print(light)
(a + light).draw_vertex(screen)

# Observer vector declaration (purple)
observer = (cam.origin - a).normalized()
observer.color = color.Color("purple")
print("observer:")
print(observer)

# Reflected vector declaration (red)


light:
Vector(0.0, 0.7071067811865475, 0.7071067811865475, green)
observer:
Vector(-0.9314928656652446, 0.23287321641631115, 0.27944785969957336, purple)


TypeError: unsupported operand type(s) for *: 'Vector' and 'int'

In [ ]:
# Cube delcaration
# cube = Cube(2)
# cube.draw_vertices(screen)
# cube.draw_wireframe(screen)

In [ ]:
# Tkinter loop
window.mainloop()